In [ ]:
#!/usr/bin/env python3
"""
repeatable_missing_rows_finder.py

Output schema (exact order):
sale_date, salesrepname, splitsalesrep, manager, bd_agent,
f_&_i, f_&_i_2, vehiclestocknumber, first_name, last_name, lead_source
"""

import sys, csv
import pandas as pd
from pathlib import Path
from typing import Iterable, Optional

# ===================== USER CONFIG =====================
DMS_FILE = "//content/PaytonListtxt.csv"                  # e.g., DeskManager / DMS export
SHEET_FILE = "/content/August Google sheet  - Main (14).csv"  # your Google Sheet export
# ======================================================

BAD_STOCK_VALUES = {"", "NAN", "NONE", "NULL"}

OUTPUT_COLUMNS = [
    "sale_date", "salesrepname", "splitsalesrep", "manager", "bd_agent",
    "f_&_i", "f_&_i_2", "vehiclestocknumber", "first_name", "last_name", "lead_source"
]

COLUMN_VARIANTS = {
    "sale_date":        ["sale_date", "sale date", "date", "date_of_sale", "sold_date"],
    "salesrepname":     ["salesrepname", "sales_rep_name", "sales_rep", "rep", "salesperson"],
    "splitsalesrep":    ["splitsalesrep", "split_sales_rep", "splitrep", "split_salesperson"],
    "manager":          ["manager", "sales_manager", "desk_manager"],
    "bd_agent":         ["bd_agent", "bdagent", "bd", "bd_agent_name", "bd rep"],
    "f_&_i":            ["f_&_i", "f & i", "f_i", "finance_manager", "fni", "f and i"],
    "f_&_i_2":          ["f_&_i_2", "f & i 2", "f_i_2", "finance_manager_2", "fni_2", "f and i 2"],
    "vehiclestocknumber": ["vehiclestocknumber", "vehicle_stock_number", "stock_number", "stocknumber", "stock", "stock_no", "stocknum"],
    "first_name":       ["first_name", "first name", "fname", "customer_first_name"],
    "last_name":        ["last_name", "last name", "lname", "customer_last_name", "surname"],
    "lead_source":      ["lead_source", "lead source", "source", "leadsource"]
}

def normalize_header(col: str) -> str:
    return str(col).strip().lower().replace(" ", "_").replace("#", "number")

def load_norm(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = [normalize_header(c) for c in df.columns]
    return df

def find_first(df: pd.DataFrame, candidates: Iterable[str]) -> Optional[str]:
    normalized = [normalize_header(c) for c in candidates]
    df_cols = set(df.columns)
    for c in normalized:
        if c in df_cols:
            return c
    return None

def find_stock_col(cols) -> Optional[str]:
    candidates = [
        "vehiclestocknumber", "stock_number", "stocknumber",
        "vehicle_stock_number", "stock", "stock_no", "stocknum"
    ]
    normalized_cols = [normalize_header(c) for c in cols]
    for c in candidates:
        if c in normalized_cols:
            return c
    for c in normalized_cols:
        if "stock" in c:
            return c
    return None

def clean_strings(df: pd.DataFrame) -> pd.DataFrame:
    """Blank out NaNs/'nan', trim whitespace, normalize newlines, drop quote-only rows."""
    df = df.copy().where(df.notna(), "")
    for c in df.columns:
        df[c] = (
            df[c].astype(str)
                 .str.replace(r"^\s*nan\s*$", "", regex=True)
                 .str.replace("\r\n", "\n")
                 .str.replace("\r", "\n")
                 .str.strip()
        )
    # drop rows that collapse to just a single double-quote
    mask_bad = df.apply(lambda r: "".join(r.values.astype(str)).strip() == '"', axis=1)
    return df[~mask_bad]

def build_output(dms_df: pd.DataFrame, sheet_df: pd.DataFrame) -> pd.DataFrame:
    dms_stock = find_stock_col(dms_df.columns)
    sheet_stock = find_stock_col(sheet_df.columns)
    if dms_stock is None or sheet_stock is None:
        raise ValueError("Could not find a stock number column in one or both files.")

    dms_df["__stock__"] = dms_df[dms_stock].astype(str).str.upper().str.strip()
    sheet_df["__stock__"] = sheet_df[sheet_stock].astype(str).str.upper().str.strip()

    dms_df = dms_df[~dms_df["__stock__"].isin(BAD_STOCK_VALUES)]
    sheet_df = sheet_df[~sheet_df["__stock__"].isin(BAD_STOCK_VALUES)]

    sheet_keys = set(sheet_df["__stock__"])
    missing_keys = [s for s in dms_df["__stock__"].tolist() if s not in sheet_keys]
    if not missing_keys:
        return pd.DataFrame(columns=OUTPUT_COLUMNS)

    missing_rows = dms_df[dms_df["__stock__"].isin(missing_keys)].copy()
    missing_rows["__order__"] = range(len(missing_rows))

    out = pd.DataFrame(index=missing_rows.index, columns=OUTPUT_COLUMNS, dtype=object)
    for out_col in OUTPUT_COLUMNS:
        if out_col == "vehiclestocknumber":
            out[out_col] = missing_rows["__stock__"]
            continue
        src_col = find_first(missing_rows, COLUMN_VARIANTS[out_col])
        out[out_col] = missing_rows[src_col] if src_col else ""

    out["__order__"] = missing_rows["__order__"]
    out = out.sort_values("__order__", kind="stable").drop(columns="__order__")

    # force strings
    for c in OUTPUT_COLUMNS:
        out[c] = out[c].astype(str)

    return clean_strings(out)

def main():
    try:
        dms_df = load_norm(Path(DMS_FILE))
        sheet_df = load_norm(Path(SHEET_FILE))
        output_df = build_output(dms_df, sheet_df)

        # Paste-friendly TSV (optional file)
        output_df.to_csv("missing_rows.tsv", sep="\t", index=False, lineterminator="\n")

        # Clean CSV (fully quoted for Import)
        output_df.to_csv(
            "missing_rows.csv",
            index=False,
            lineterminator="\n",
            quoting=csv.QUOTE_ALL,
            escapechar="\\",
            na_rep=""
        )

        # Console preview (CSV, normal-looking)
        print(output_df.to_csv(index=False, lineterminator="\n"))
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure both files are uploaded and paths are correct.", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()


sale_date,salesrepname,splitsalesrep,manager,bd_agent,f_&_i,f_&_i_2,vehiclestocknumber,first_name,last_name,lead_source
8/20/25,Tim Bateman,,Levi Polege,,Mark Rector,,MP10009,Mark,Sidell,DEALERS WEBSITE
8/20/25,Christian Mack,Mark Rector,Suna Faitarouny,,Brent Beck,,MP10049,Noah,Acker,
8/20/25,Daniel Lande,,Suna Faitarouny,,Mark Rector,,MP10093,Guadelupe,Alvarado,DEALERS WEBSITE
8/20/25,Brandon Beltran,,Mark Rector,,,,MP10218,Wayne,Woodgate,
8/20/25,Trey Flores,,Suna Faitarouny,,Brent Beck,,MP10019A,Isaiah,Sanchez,AUTOTRADER
8/21/25,,,,,,,MP9442,,,
8/21/25,,,,,,,MP10191A,,,
8/21/25,Jordan Lecky,,Suna Faitarouny,,Yesenia Felix,,MP10274,Jessyka,Johnson,REPEAT



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!/usr/bin/env python3
"""
missing_stock_numbers_analyzer.py

Analyzes two CSV files to find stock numbers present in one but not the other,
with an added filter for 'Employee' and 'Whole sale' entries.
"""

import pandas as pd
from typing import List, Set, Optional, Tuple

# --- Helper Functions for Data Loading and Normalization ---

def normalize_header(col: str) -> str:
    """Normalizes a column header for consistent access."""
    return str(col).strip().lower().replace(" ", "_").replace("#", "number")

def find_stock_col(cols: List[str]) -> Optional[str]:
    """
    Tries to find the stock number column from a list of headers.
    Returns the normalized column name if found, otherwise None.
    """
    candidates = [
        "vehiclestocknumber", "stock_number", "stocknumber",
        "vehicle_stock_number", "stock", "stock_no", "stocknum",
        "stock_number"
    ]
    for c in candidates:
        if c in cols:
            return c
    return None

def load_data(file_path: str) -> Optional[pd.DataFrame]:
    """Loads a CSV file and normalizes its column headers."""
    try:
        df = pd.read_csv(file_path)
        df.columns = [normalize_header(c) for c in df.columns]
        return df
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while processing '{file_path}': {e}")
        return None

def get_clean_stocks_set(df: pd.DataFrame) -> Set[str]:
    """
    Finds the stock number column in a DataFrame and returns a
    set of unique, normalized stock numbers.
    """
    stock_col = find_stock_col(df.columns)
    if stock_col is None:
        print(f"Warning: Could not find a stock number column in a dataframe.")
        return set()

    # Extract, normalize, and return unique stock numbers
    # Filter out junk/empty values
    stocks = df[stock_col].astype(str).str.upper().str.strip()
    cleaned_stocks = stocks[stocks.str.len() > 0]
    return set(cleaned_stocks)

# --- Main Analysis Logic ---

def analyze_missing_stocks(file1: str, file2: str):
    """
    Compares the stock numbers between two files and prints the results,
    with a filter on the first file.
    """
    print("--- Analyzing Missing Stock Numbers ---")
    print(f"File 1 (Filtered): {file1}")
    print(f"File 2: {file2}")
    print("-" * 35)

    # Load dataframes
    df1 = load_data(file1)
    df2 = load_data(file2)

    if df1 is None or df2 is None:
        print("Could not load one or both files. Cannot compare.")
        return

    # --- ADDED FILTER LOGIC ---
    # Filter the first DataFrame to exclude 'employee' and 'whole sale' leads
    source_col = 'lead_source'
    if source_col in df1.columns:
        original_count = len(df1)
        df1 = df1[~df1[source_col].astype(str).str.lower().isin(['employee', 'whole sale'])]
        filtered_count = len(df1)
        print(f"Filter applied: Removed {original_count - filtered_count} entries from '{file1}'.")
    # --- END ADDED FILTER LOGIC ---

    # Get the clean sets of stock numbers
    file1_stocks = get_clean_stocks_set(df1)
    file2_stocks = get_clean_stocks_set(df2)

    if not file1_stocks and not file2_stocks:
        print("No stock numbers could be extracted from either file. Cannot compare.")
        return

    # Find stocks in file1 but not in file2
    missing_in_file2 = file1_stocks - file2_stocks

    # Find stocks in file2 but not in file1
    missing_in_file1 = file2_stocks - file1_stocks

    # Print the results
    print(f"\nStocks in '{file1}' but MISSING from '{file2}':")
    if missing_in_file2:
        for stock in sorted(list(missing_in_file2)):
            print(f"- {stock}")
    else:
        print("None.")

    print(f"\nStocks in '{file2}' but MISSING from '{file1}':")
    if missing_in_file1:
        for stock in sorted(list(missing_in_file1)):
            print(f"- {stock}")
    else:
        print("None.")

    print("\n--- Analysis Complete ---")

# --- Entry Point ---

if __name__ == "__main__":
    google_sheet_file = "/content/August Google sheet  - Main (15).csv"
    sales_detail_file = "/content/SalesDetail_2025-08-21.csv"
    analyze_missing_stocks(google_sheet_file, sales_detail_file)

--- Analyzing Missing Stock Numbers ---
File 1 (Filtered): /content/August Google sheet  - Main (15).csv
File 2: /content/SalesDetail_2025-08-21.csv
-----------------------------------
Filter applied: Removed 15 entries from '/content/August Google sheet  - Main (15).csv'.

Stocks in '/content/August Google sheet  - Main (15).csv' but MISSING from '/content/SalesDetail_2025-08-21.csv':
- MP10049
- MP10218
- MP9698

Stocks in '/content/SalesDetail_2025-08-21.csv' but MISSING from '/content/August Google sheet  - Main (15).csv':
None.

--- Analysis Complete ---


In [ ]:
#!/usr/bin/env python3
"""
sales_mismatch_analyzer.py

Task:
- Compares records between a primary CRM file and a DMS file.
- Uses a consistent 'stock_number' field for comparison.
- Identifies and reports specific mismatches for both primary and split sales reps.

This script is designed for easy use in Google Colab with minimal changes.
"""
import sys
import pandas as pd
from pathlib import Path
from typing import List, Set, Optional, Dict, Any

# =============================================================================
#
#   USER CONFIGURATION SECTION
#   Change these two variables to point to your input files.
#
# =============================================================================

# Name of the primary CRM sales file (e.g., SalesDetail)
CRM_FILE = "/content/SalesDetail_2025-08-16.csv"

# Name of the DMS file (e.g., PaytonListtxt or a Google Sheet export)
DMS_FILE = "/content/PaytonListtxt.csv"

# =============================================================================
#
#   CORE SCRIPT LOGIC (DO NOT CHANGE BELOW THIS LINE)
#
# =============================================================================

BAD_STOCK_VALUES = {"", "NAN", "NONE", "NULL"}
STOCK_COL_NAME = "stock_number"
SALESREP_COL_NAME = "salesrepname"
SPLITSALESREP_COL_NAME = "splitsalesrep"

def normalize_header(col: str) -> str:
    """Normalizes a column header for consistent access."""
    return str(col).strip().lower().replace(" ", "_").replace("#", "number")

def find_and_standardize_cols(df: pd.DataFrame, original_col_names: List[str], target_name: str) -> pd.DataFrame:
    """
    Finds a column from a list of candidates and renames it to a standard name.
    """
    normalized_cols = [normalize_header(c) for c in df.columns]

    found_col_norm = None
    for c in original_col_names:
        if c in normalized_cols:
            found_col_norm = c
            break

    if found_col_norm:
        original_col_name = df.columns[normalized_cols.index(found_col_norm)]
        if original_col_name != target_name:
            df = df.rename(columns={original_col_name: target_name})
        return df
    else:
        # If column not found, add a blank column to avoid KeyError later
        df[target_name] = "MISSING"
        return df

def load_clean(path: str) -> pd.DataFrame:
    """
    Loads a CSV, cleans headers, and standardizes column names for analysis.
    """
    try:
        df = pd.read_csv(path)
        # Normalize all column names first
        df.columns = [normalize_header(c) for c in df.columns]

        # Standardize key columns
        df = find_and_standardize_cols(df, ["stock_number", "vehiclestocknumber", "stock"], STOCK_COL_NAME)
        df = find_and_standardize_cols(df, ["sales_rep", "salesrepname"], SALESREP_COL_NAME)
        df = find_and_standardize_cols(df, ["split_sales_rep", "splitsalesrep"], SPLITSALESREP_COL_NAME)

        # Clean the stock number data
        df[STOCK_COL_NAME] = df[STOCK_COL_NAME].astype(str).str.upper().str.strip()
        return df.fillna("MISSING")
    except FileNotFoundError:
        print(f"Error: The file '{path}' was not found.", file=sys.stderr)
        raise
    except KeyError as e:
        print(f"Error: {e}. One of the files may be missing a required column.", file=sys.stderr)
        raise

# Drop duplicates
def drop_duplicates_and_set_index(df: pd.DataFrame, subset_col: str) -> Dict[str, Any]:
    df = df.drop_duplicates(subset=subset_col, keep="first")
    return df.set_index(subset_col).to_dict("index")

def analyze_mismatches(crm_dict: Dict[str, Any], dms_dict: Dict[str, Any]):
    mismatches = []

    for stock, crm_row in crm_dict.items():
        dms_row = dms_dict.get(stock)

        if dms_row is None:
            continue

        # Get specific values for each field
        crm_salesrep = str(crm_row.get(SALESREP_COL_NAME, "MISSING")).strip().lower()
        dms_salesrep = str(dms_row.get(SALESREP_COL_NAME, "MISSING")).strip().lower()

        crm_splitsalesrep = str(crm_row.get(SPLITSALESREP_COL_NAME, "MISSING")).strip().lower()
        dms_splitsalesrep = str(dms_row.get(SPLITSALESREP_COL_NAME, "MISSING")).strip().lower()

        # New logic: Check if the *set* of names is different.
        crm_reps_set = {crm_salesrep, crm_splitsalesrep} - {"missing", ""}
        dms_reps_set = {dms_salesrep, dms_splitsalesrep} - {"missing", ""}

        if crm_reps_set != dms_reps_set:
            mismatches.append({
                "stock_number": stock,
                "Field": "Sales Reps",
                "Salesrep value in CRM": crm_salesrep,
                "Salesrep value in DMS": dms_salesrep,
                "SplitSalesrep value in CRM": crm_splitsalesrep,
                "SplitSalesrep value in DMS": dms_splitsalesrep,
            })

    mismatch_df = pd.DataFrame(mismatches)
    print("\n--- Sales Rep Mismatch Report ---")
    if not mismatch_df.empty:
        print(mismatch_df.to_string(index=False))
    else:
        print("No mismatches found between the CRM and DMS files.")
    print("-----------------------------------")


if __name__ == "__main__":
    try:
        # Load and clean both files
        crm_df = load_clean(CRM_FILE)
        dms_df = load_clean(DMS_FILE)

        # Convert to dictionaries for efficient lookup
        crm_dict = drop_duplicates_and_set_index(crm_df, STOCK_COL_NAME)
        dms_dict = drop_duplicates_and_set_index(dms_df, STOCK_COL_NAME)

        # Run the analysis
        analyze_mismatches(crm_dict, dms_dict)

    except FileNotFoundError as e:
        print(f"ERROR: {e}. Please ensure the input files are uploaded to Colab.")
    except KeyError as e:
        print(f"ERROR: {e}. One of the files may be missing a required column.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)


--- Sales Rep Mismatch Report ---
stock_number      Field Salesrep value in CRM Salesrep value in DMS SplitSalesrep value in CRM SplitSalesrep value in DMS
      MP9807 Sales Reps           trey flores         robert emmons              robert emmons                    missing
      MP8991 Sales Reps         jesus galindo          will deguire                    missing              robert emmons
     MP10234 Sales Reps           tim bateman           tim bateman            brandon beltran                    missing
-----------------------------------


In [ ]:
#!/usr/bin/env python3
"""
lead_source_mismatch_analyzer.py

Task:
- Compares the 'lead_source' field across three CSV files:
  - a primary sales detail log
  - a Google Sheet export
  - a PaytonList DMS export
- Standardizes similar lead sources (e.g., 'Facebook Marketplace' -> 'Facebook')
  to avoid false mismatches.
- Identifies and reports any records where the standardized 'lead_source' value
  does not match across all three files.

This script is designed for easy use in Google Colab with minimal changes.
"""

import sys
import pandas as pd
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple

# =============================================================================
#
#   USER CONFIGURATION SECTION
#   Change these three variables to point to your input files.
#
# =============================================================================

# Name of the primary sales file (e.g., SalesDetail)
SALES_FILE = "/content/SalesDetail_2025-08-16.csv"

# Name of the Google Sheet file
GOOGLE_SHEET_FILE = "/content/August Google sheet  - Main (11).csv"

# Name of the PaytonList DMS file
PAYTON_LIST_FILE = "/content/PaytonListtxt.csv"

# Lead source keywords to standardize.
# The key is the standardized root name, and the list contains keywords to match.
SOURCE_MAPPING = {
    "cargurus": ["cargurus", "cargurus - digital deal", "cargurus - credit app", "cargurus - soft pull"],
    "facebook": ["facebook", "facebook market place", "facebook marketplace"],
    "autotrader": ["autotrader", "autotrader.com", "autotrader group"],
    "cars.com": ["cars.com", "cars.com was source"],
    "dealers website": ["dealer website", "dealers website", "dealers website", "vin", "vinsolutions - finance application"],
    "carfax": ["carfax", "carfax, inc"],
    "truecar": ["truecar", "truecar group"],
    "repeat": ["repeat", "referral", "referral/repeat"],
    "walk-in": ["walk-in"],
}

# =============================================================================
#
#   CORE SCRIPT LOGIC (DO NOT CHANGE BELOW THIS LINE)
#
# =============================================================================

# Candidate columns for stock number
POSSIBLE_STOCK_COLS = [
    "vehiclestocknumber", "stock_number", "stock"
]

# Strings to treat as empty
BAD_SENTINELS = {"", "nan", "none", "null", "na", "n/a", "missing"}

def normalize_header(col: str) -> str:
    """Normalizes a column header for consistent access."""
    return str(col).strip().lower().replace(" ", "_").replace("#", "number")

def find_column(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """Finds a column name in a DataFrame from a list of candidates."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def standardize_source(source: str) -> str:
    """Standardizes a lead source string based on a predefined mapping."""
    source = str(source).strip().lower()
    for root, keywords in SOURCE_MAPPING.items():
        if any(keyword in source for keyword in keywords):
            return root
    return source  # Return original if no match found

def load_and_clean(path: str) -> Tuple[pd.DataFrame, str]:
    """
    Loads a CSV file, standardizes column names, and cleans key fields.
    """
    try:
        df = pd.read_csv(path)
        # Standardize headers to lowercase with underscores
        df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_").str.replace("-", "_")

        # Find and standardize the stock number column
        stock_col = find_column(df, POSSIBLE_STOCK_COLS)
        if stock_col is None:
             raise KeyError("Could not find a stock number column in the file.")

        df[stock_col] = df[stock_col].astype(str).str.upper().str.strip()

        # Clean and standardize the lead source column
        lead_source_col = find_column(df, ["lead_source", "lead_source_group"])
        if lead_source_col is None:
            df["lead_source"] = "MISSING"
        else:
            df["lead_source"] = df[lead_source_col].fillna("MISSING").apply(standardize_source)

        return df, stock_col
    except FileNotFoundError:
        print(f"Error: The file '{path}' was not found.", file=sys.stderr)
        sys.exit(1)
    except KeyError as e:
        print(f"Error: A required column was not found in '{path}': {e}", file=sys.stderr)
        sys.exit(1)

def drop_duplicates_and_set_index(df: pd.DataFrame, subset_col: str) -> Dict[str, Any]:
    """
    Drops duplicate rows and converts a DataFrame to a dictionary for lookup.
    """
    df = df.drop_duplicates(subset=subset_col, keep="first")
    return df.set_index(subset_col).to_dict("index")

def analyze_lead_source_mismatches():
    """
    Main function to load files, compare lead sources, and report mismatches.
    """
    # Load and clean all three data files
    sales_df, sales_stock_col = load_and_clean(SALES_FILE)
    google_df, google_stock_col = load_and_clean(GOOGLE_SHEET_FILE)
    payton_df, payton_stock_col = load_and_clean(PAYTON_LIST_FILE)

    # Convert to lookup dictionaries
    sales_dict = drop_duplicates_and_set_index(sales_df, sales_stock_col)
    google_dict = drop_duplicates_and_set_index(google_df, google_stock_col)
    payton_dict = drop_duplicates_and_set_index(payton_df, payton_stock_col)

    mismatches = []

    # Iterate through the sales data to check for discrepancies
    for stock, sales_row in sales_dict.items():
        source_sold = sales_row.get("lead_source", "MISSING")
        google_row = google_dict.get(stock)
        payton_row = payton_dict.get(stock)

        if not google_row or not payton_row:
            continue

        source_google = google_row.get("lead_source", "MISSING")
        source_payton = payton_row.get("lead_source", "MISSING")

        if (source_sold != source_google) or (source_sold != source_payton):
            mismatches.append({
                "vehiclestocknumber": stock,
                "field": "lead_source",
                "SalesDetail": source_sold,
                "GoogleSheet": source_google,
                "PaytonList": source_payton
            })

    mismatch_df = pd.DataFrame(mismatches)

    print("\n--- Lead Source Mismatch Report ---")
    if not mismatch_df.empty:
        print(mismatch_df.to_string(index=False))
    else:
        print("No lead source mismatches found across the three files.")
    print("-----------------------------------")


if __name__ == "__main__":
    analyze_lead_source_mismatches()



--- Lead Source Mismatch Report ---
vehiclestocknumber       field          SalesDetail GoogleSheet PaytonList
            MP9655 lead_source              truecar    true car   true car
           MP10040 lead_source          auto fusion  autofusion autofusion
           MP10128 lead_source               carfax     missing     carfax
           MP10197 lead_source      dealers website    cargurus   cargurus
            MP9790 lead_source engage to sell sales      engage     engage
            MP9980 lead_source              truecar    true car   true car
           MP10193 lead_source      dealers website      repeat     repeat
            MP9983 lead_source             cargurus     walk-in    walk-in
           MP10065 lead_source             cargurus    employee   cargurus
           MP10121 lead_source             facebook     missing    missing
           MP8793A lead_source          auto fusion    cargurus   cargurus
-----------------------------------


In [ ]:
#!/usr/bin/env python3
"""
manager_mismatch_analyzer.py

Task:
- Compares records between a primary CRM file and a DMS file.
- Identifies and reports mismatches in manager and BD agent names.
- Uses a consistent 'vehiclestocknumber' field for comparison.

This script is designed for easy use in Google Colab with minimal changes.
"""
import sys
import pandas as pd
from pathlib import Path
from typing import List, Set, Optional, Dict, Any

# =============================================================================
#
#   USER CONFIGURATION SECTION
#   Change these two variables to point to your input files.
#
# =============================================================================

# Name of the primary CRM file (e.g., SalesDetail)
CRM_FILE = "/content/SalesDetail_2025-08-13 (4).csv"

# Name of the DMS file (e.g., PaytonListtxt or a Google Sheet export)
DMS_FILE = "/content/August Google sheet  - Main (6).csv"

# =============================================================================
#
#   CORE SCRIPT LOGIC (DO NOT CHANGE BELOW THIS LINE)
#
# =============================================================================

BAD_STOCK_VALUES = {"", "NAN", "NONE", "NULL"}
STOCK_COL_NAME = "vehiclestocknumber"

def normalize_header(col: str) -> str:
    """Normalizes a column header for consistent access."""
    return str(col).strip().lower().replace(" ", "_").replace("#", "number")

def find_and_standardize_cols(df: pd.DataFrame, candidates: List[str], target_name: str) -> pd.DataFrame:
    """
    Finds a column from a list of candidates and renames it to a standard name.
    """
    normalized_cols = [normalize_header(c) for c in df.columns]

    found_col_norm = None
    for c in candidates:
        if c in normalized_cols:
            found_col_norm = c
            break

    if found_col_norm:
        original_col_name = df.columns[normalized_cols.index(found_col_norm)]
        if original_col_name != target_name:
            df = df.rename(columns={original_col_name: target_name})
        return df
    else:
        raise KeyError(f"Could not find a required column. Tried candidates: {candidates}")

def load_and_clean(path: str) -> pd.DataFrame:
    """
    Loads a CSV file, standardizes column names, and cleans key fields.
    """
    try:
        df = pd.read_csv(path)
        # Standardize headers to lowercase with underscores
        df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")

        # Standardize the stock number column name
        df = find_and_standardize_cols(df, ["vehiclestocknumber", "stock_number", "stock"], STOCK_COL_NAME)

        # Standardize manager and BD agent column names
        df = find_and_standardize_cols(df, ["manager", "manager_name"], "manager")
        df = find_and_standardize_cols(df, ["bd_agent"], "bd_agent")

        # Clean up the stock number and manager columns
        df[STOCK_COL_NAME] = df[STOCK_COL_NAME].astype(str).str.upper().str.strip()
        df = df.fillna("MISSING")

        return df
    except FileNotFoundError:
        print(f"Error: The file '{path}' was not found.", file=sys.stderr)
        sys.exit(1)
    except KeyError as e:
        print(f"Error: A required column was not found in '{path}': {e}", file=sys.stderr)
        sys.exit(1)

def normalize_people(row: Dict[str, Any], cols: List[str]) -> Set[str]:
    """
    Extracts and normalizes names from specified columns into a set.
    """
    return {
        str(row.get(col, "MISSING")).strip().lower()
        for col in cols if str(row.get(col, "MISSING")).strip().lower() not in ["missing", ""]
    }

def drop_duplicates_and_set_index(df: pd.DataFrame, subset_col: str) -> Dict[str, Any]:
    """
    Drops duplicate rows and converts a DataFrame to a dictionary for lookup.
    """
    df = df.drop_duplicates(subset=subset_col, keep="first")
    return df.set_index(subset_col).to_dict("index")

def analyze_manager_mismatches():
    """
    Main function to load files, compare manager/BD agent, and report mismatches.
    """
    try:
        # Load and clean both data files
        crm_df = load_and_clean(CRM_FILE)
        dms_df = load_and_clean(DMS_FILE)

        # Convert the DMS dataframe to a lookup dictionary
        dms_dict = drop_duplicates_and_set_index(dms_df, STOCK_COL_NAME)

        mismatches = []

        for _, crm_row in crm_df.iterrows():
            stock = crm_row[STOCK_COL_NAME]
            dms_row = dms_dict.get(stock)

            # Skip if the record isn't in both files
            if dms_row is None:
                continue

            # Extract and normalize manager/BD agent names from both sources
            crm_managers = normalize_people(crm_row, ["manager", "bd_agent"])
            dms_managers = normalize_people(dms_row, ["manager", "bd_agent"])

            # Compare the sets of names
            if crm_managers != dms_managers:
                mismatches.append({
                    "stock_number": stock,
                    "field": "Managers",
                    "value_in_crm": sorted(list(crm_managers)) or ["MISSING"],
                    "value_in_dms": sorted(list(dms_managers)) or ["MISSING"],
                })

        mismatch_df = pd.DataFrame(mismatches)

        print("\n--- Manager Mismatch Report ---")
        if not mismatch_df.empty:
            print(mismatch_df.to_string(index=False))
        else:
            print("No manager mismatches found between the CRM and DMS files.")
        print("-----------------------------------")

    except Exception as e:
        print(f"An error occurred: {e}", file=sys.stderr)


if __name__ == "__main__":
    analyze_manager_mismatches()


--- Manager Mismatch Report ---
stock_number    field  value_in_crm     value_in_dms
     MP10150 Managers [levi polege] [jamie sheridan]
-----------------------------------


In [ ]:
#!/usr/bin/env python3
"""
attribute_sales_compare.py

Task:
- Loads two CSVs (CRM and DMS), cleans columns, computes sales attribution
  per person using simple rules, and outputs a side-by-side comparison.

Attribution rules:
- Full sale (1.0): exactly one of {manager, bd_agent} is present.
- Split sale (0.5 each): both present.
- Unassigned (1.0): neither present -> credited to "unassigned".

This script is designed for easy use in Google Colab with minimal changes.
"""

import sys
import pandas as pd
from pathlib import Path
from collections import defaultdict
from typing import Dict, Any, List, Optional, Tuple

# =============================================================================
#
#   USER CONFIGURATION SECTION
#   Change these variables to point to your input files.
#
# =============================================================================

# Name of the CRM file
CRM_FILE = "/content/August Google sheet  - Main (8).csv"

# Name of the DMS file
DMS_FILE = "/content/SalesDetail_2025-08-13 (6).csv"

# Directory to save the output files
OUT_DIR = "out"

# =============================================================================
#
#   CORE SCRIPT LOGIC (DO NOT CHANGE BELOW THIS LINE)
#
# =============================================================================

# Candidate columns
POSSIBLE_MANAGER_COLS = [
    "manager", "sales_manager", "salesmanager", "desk_manager", "deskmanager"
]
POSSIBLE_BD_COLS = [
    "bd_agent", "bdagent", "business_development", "bd", "internet_agent", "internet_manager"
]

# Strings to treat as empty
BAD_SENTINELS = {"", "nan", "none", "null", "na", "n/a", "missing"}

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    """Normalizes DataFrame headers for consistent access."""
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

def find_column(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """Finds a column name in a DataFrame from a list of candidates."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def cleaned_series(s: Optional[pd.Series]) -> pd.Series:
    """Cleans a Series of strings for consistent processing."""
    if s is None:
        return pd.Series([], dtype=str)
    s = s.fillna("")
    s = s.astype(str).str.strip().str.lower()
    s = s.apply(lambda x: "" if x in BAD_SENTINELS else x)
    return s

def load_and_prepare(path: Path) -> Tuple[pd.DataFrame, str, str]:
    """Loads a CSV, cleans headers, and identifies manager/BD agent columns."""
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")

    df = pd.read_csv(path)
    df = normalize_headers(df)

    manager_col = find_column(df, POSSIBLE_MANAGER_COLS)
    bd_col = find_column(df, POSSIBLE_BD_COLS)

    # If missing, create empty columns to keep logic simple
    if manager_col is None:
        manager_col = "__manager_missing__"
        df[manager_col] = ""
    if bd_col is None:
        bd_col = "__bd_missing__"
        df[bd_col] = ""

    df[manager_col] = cleaned_series(df[manager_col])
    df[bd_col] = cleaned_series(df[bd_col])

    return df, manager_col, bd_col

def tally_attribution(df: pd.DataFrame, manager_col: str, bd_col: str) -> Dict[str, float]:
    """Counts attributed sales based on defined rules."""
    counts: defaultdict[str, float] = defaultdict(float)
    for m, b in zip(df[manager_col], df[bd_col]):
        has_m = bool(m)
        has_b = bool(b)
        if has_m and not has_b:
            counts[m] += 1.0
        elif has_b and not has_m:
            counts[b] += 1.0
        elif has_m and has_b:
            counts[m] += 0.5
            counts[b] += 0.5
        else:
            counts["unassigned"] += 1.0
    return dict(counts)

def dict_to_df(d: Dict[str, float], count_label: str) -> pd.DataFrame:
    """Converts a dictionary of counts into a sorted DataFrame."""
    return (
        pd.DataFrame(list(d.items()), columns=["person", count_label])
        .sort_values(by=count_label, ascending=False)
        .reset_index(drop=True)
    )

def compare_side_by_side(crm_df: pd.DataFrame, dms_df: pd.DataFrame) -> pd.DataFrame:
    """Merges CRM and DMS data to show a side-by-side comparison."""
    merged = pd.merge(crm_df, dms_df, on="person", how="outer").fillna(0.0)
    merged["diff_dms_minus_crm"] = merged["dms_count"] - merged["crm_count"]
    merged = merged.sort_values(by=["diff_dms_minus_crm", "person"], ascending=[False, True]).reset_index(drop=True)
    return merged

def main():
    try:
        crm_path = Path(CRM_FILE)
        dms_path = Path(DMS_FILE)
        outdir = Path(OUT_DIR)
        outdir.mkdir(parents=True, exist_ok=True)

        # Load + tally CRM
        crm_df, mcol_crm, bcol_crm = load_and_prepare(crm_path)
        crm_counts = tally_attribution(crm_df, mcol_crm, bcol_crm)
        crm_tbl = dict_to_df(crm_counts, "crm_count")

        # Load + tally DMS
        dms_df, mcol_dms, bcol_dms = load_and_prepare(dms_path)
        dms_counts = tally_attribution(dms_df, mcol_dms, bcol_dms)
        dms_tbl = dict_to_df(dms_counts, "dms_count")

        # Compare
        comparison = compare_side_by_side(crm_tbl, dms_tbl)

        # Define output file paths
        crm_out = outdir / "crm_attribution.csv"
        dms_out = outdir / "dms_attribution.csv"
        comp_out = outdir / "comparison_side_by_side.csv"

        # Save output files
        crm_tbl.to_csv(crm_out, index=False)
        dms_tbl.to_csv(dms_out, index=False)
        comparison.to_csv(comp_out, index=False)

        # Console summary
        print("\n=== CRM Attribution (top 10) ===")
        print(crm_tbl.head(10).to_string(index=False))
        print("\n=== DMS Attribution (top 10) ===")
        print(dms_tbl.head(10).to_string(index=False))
        print("\n=== Side-by-Side Comparison (top 20 by diff) ===")
        print(comparison.head(20).to_string(index=False))
        print(f"\nColumns used -> CRM: manager='{mcol_crm}', bd_agent='{bcol_crm}' | DMS: manager='{mcol_dms}', bd_agent='{bcol_dms}'")
        print(f"Wrote files:\n- {crm_out}\n- {dms_out}\n- {comp_out}")

    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure the input files are uploaded and the names match the configuration.", file=sys.stderr)
    except KeyError as e:
        print(f"Error: A required column was not found in one of the files: {e}", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()




=== CRM Attribution (top 10) ===
         person  crm_count
suna faitarouny       48.0
 jamie sheridan       13.5
    levi polege       12.5
     unassigned        5.0

=== DMS Attribution (top 10) ===
         person  dms_count
suna faitarouny       46.0
 jamie sheridan       13.5
    levi polege       12.5

=== Side-by-Side Comparison (top 20 by diff) ===
         person  crm_count  dms_count  diff_dms_minus_crm
 jamie sheridan       13.5       13.5                 0.0
    levi polege       12.5       12.5                 0.0
suna faitarouny       48.0       46.0                -2.0
     unassigned        5.0        0.0                -5.0

Columns used -> CRM: manager='manager', bd_agent='bd_agent' | DMS: manager='manager', bd_agent='bd_agent'
Wrote files:
- out/crm_attribution.csv
- out/dms_attribution.csv
- out/comparison_side_by_side.csv


In [ ]:
#!/usr/bin/env python3
"""
mismatch_stock_finder.py

Task:
- Compares sales attribution between a CRM file and a DMS file.
- Identifies and reports specific stock numbers where manager/BD agent
  attribution differs between the two files.

This script is designed for easy use in Google Colab with minimal changes.
"""
import sys
import pandas as pd
from pathlib import Path
from typing import Dict, Any, List, Optional, Set, Tuple

# =============================================================================
#
#   USER CONFIGURATION SECTION
#   Change these variables to point to your input files.
#
# =============================================================================

# Name of the CRM file
CRM_FILE = "/content/SalesDetail_2025-08-13 (6).csv"

# Name of the DMS file
DMS_FILE = "/content/August Google sheet  - Main (8).csv"

# =============================================================================
#
#   CORE SCRIPT LOGIC (DO NOT CHANGE BELOW THIS LINE)
#
# =============================================================================

# Candidate columns
POSSIBLE_MANAGER_COLS = [
    "manager", "sales_manager", "salesmanager", "desk_manager", "deskmanager"
]
POSSIBLE_BD_COLS = [
    "bd_agent", "bdagent", "business_development", "bd", "internet_agent", "internet_manager"
]

# Strings to treat as empty
BAD_SENTINELS = {"", "nan", "none", "null", "na", "n/a", "missing"}

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    """Normalizes DataFrame headers for consistent access."""
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

def find_column(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """Finds a column name in a DataFrame from a list of candidates."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def cleaned_series(s: Optional[pd.Series]) -> pd.Series:
    """Cleans a Series of strings for consistent processing."""
    if s is None:
        return pd.Series([], dtype=str)
    s = s.fillna("")
    s = s.astype(str).str.strip().str.lower()
    s = s.apply(lambda x: "" if x in BAD_SENTINELS else x)
    return s

def load_and_prepare(path: Path) -> Tuple[pd.DataFrame, str, str, str]:
    """
    Loads a CSV, cleans headers, and identifies key columns.
    Returns the DataFrame and the identified column names.
    """
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")

    df = pd.read_csv(path)
    df = normalize_headers(df)

    stock_col = find_column(df, ["vehiclestocknumber", "stock_number", "stock"])
    manager_col = find_column(df, POSSIBLE_MANAGER_COLS)
    bd_col = find_column(df, POSSIBLE_BD_COLS)

    if stock_col is None:
        raise KeyError(f"Could not find a stock number column in file: {path}")

    # If missing, create empty columns to keep logic simple
    if manager_col is None:
        manager_col = "__manager_missing__"
        df[manager_col] = ""
    if bd_col is None:
        bd_col = "__bd_missing__"
        df[bd_col] = ""

    df[stock_col] = cleaned_series(df[stock_col])
    df[manager_col] = cleaned_series(df[manager_col])
    df[bd_col] = cleaned_series(df[bd_col])

    return df, stock_col, manager_col, bd_col

def get_attribution_set(row: Dict[str, Any], manager_col: str, bd_col: str) -> Set[str]:
    """
    Extracts and cleans manager/BD agent names from a row into a set.
    """
    names = {row[manager_col], row[bd_col]}
    return {name for name in names if name and name not in BAD_SENTINELS}

def main():
    try:
        crm_path = Path(CRM_FILE)
        dms_path = Path(DMS_FILE)

        # Load + prep both files
        crm_df, crm_stock, crm_mgr_col, crm_bd_col = load_and_prepare(crm_path)
        dms_df, dms_stock, dms_mgr_col, dms_bd_col = load_and_prepare(dms_path)

        # Create lookup dictionaries for efficient comparison
        crm_dict = crm_df.drop_duplicates(subset=crm_stock).set_index(crm_stock).T.to_dict('dict')
        dms_dict = dms_df.drop_duplicates(subset=dms_stock).set_index(dms_stock).T.to_dict('dict')

        mismatches = []

        # Iterate over all stock numbers found in the CRM file
        for stock_num, crm_row in crm_dict.items():
            if stock_num in dms_dict:
                dms_row = dms_dict[stock_num]

                crm_reps = get_attribution_set(crm_row, crm_mgr_col, crm_bd_col)
                dms_reps = get_attribution_set(dms_row, dms_mgr_col, dms_bd_col)

                # Check for a mismatch in the combined set of attributed reps
                if crm_reps != dms_reps:
                    mismatches.append({
                        "stock_number": stock_num,
                        "crm_attribution": sorted(list(crm_reps)) if crm_reps else "unassigned",
                        "dms_attribution": sorted(list(dms_reps)) if dms_reps else "unassigned",
                    })

        mismatch_df = pd.DataFrame(mismatches)

        print("\n=== Manager/BD Agent Mismatches by Stock Number ===")
        if not mismatch_df.empty:
            # Format the list of reps for better readability
            mismatch_df['crm_attribution'] = mismatch_df['crm_attribution'].apply(lambda x: ", ".join(x).title() if isinstance(x, list) else x.title())
            mismatch_df['dms_attribution'] = mismatch_df['dms_attribution'].apply(lambda x: ", ".join(x).title() if isinstance(x, list) else x.title())
            print(mismatch_df.to_string(index=False))
        else:
            print("No discrepancies found between CRM and DMS attribution.")
        print("-----------------------------------------------------")

    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure the input files are uploaded and the names match the configuration.", file=sys.stderr)
    except KeyError as e:
        print(f"Error: A required column was not found in one of the files: {e}", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()



=== Manager/BD Agent Mismatches by Stock Number ===
No discrepancies found between CRM and DMS attribution.
-----------------------------------------------------


In [ ]:
#!/usr/bin/env python3
"""
single_manager_audit_tool.py

Task:
- Audits a specific manager's sales attribution between a CRM and a DMS file.
- Identifies and reports stock numbers where the manager's attribution is either
  missing or misattributed between the two files.

This script is designed for easy use in Google Colab with minimal changes.
"""
import sys
import pandas as pd
from pathlib import Path
from typing import Dict, Any, List, Optional, Set, Tuple

# =============================================================================
#
#   USER CONFIGURATION SECTION
#   Change these variables to point to your input files and the manager to audit.
#
# =============================================================================

# Name of the CRM file
CRM_FILE = "/content/SalesDetail_2025-08-13 (6).csv"

# Name of the DMS file
DMS_FILE = "/content/August Google sheet  - Main (8).csv"

# The name of the manager to audit (case-insensitive)
TARGET_MANAGER = "suna faitarouny"

# =============================================================================
#
#   CORE SCRIPT LOGIC (DO NOT CHANGE BELOW THIS LINE)
#
# =============================================================================

# Candidate columns
POSSIBLE_MANAGER_COLS = [
    "manager", "sales_manager", "salesmanager", "desk_manager", "deskmanager"
]
POSSIBLE_BD_COLS = [
    "bd_agent", "bdagent", "business_development", "bd", "internet_agent", "internet_manager"
]

# Strings to treat as empty
BAD_SENTINELS = {"", "nan", "none", "null", "na", "n/a", "missing"}

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    """Normalizes DataFrame headers for consistent access."""
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

def find_column(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """Finds a column name in a DataFrame from a list of candidates."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def cleaned_series(s: Optional[pd.Series]) -> pd.Series:
    """Cleans a Series of strings for consistent processing."""
    if s is None:
        return pd.Series([], dtype=str)
    s = s.fillna("")
    s = s.astype(str).str.strip().str.lower()
    s = s.apply(lambda x: "" if x in BAD_SENTINELS else x)
    return s

def load_and_prepare(path: Path) -> Tuple[pd.DataFrame, str, str, str]:
    """
    Loads a CSV, cleans headers, and identifies key columns.
    Returns the DataFrame and the identified column names.
    """
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")

    df = pd.read_csv(path)
    df = normalize_headers(df)

    stock_col = find_column(df, ["vehiclestocknumber", "stock_number", "stock"])
    manager_col = find_column(df, POSSIBLE_MANAGER_COLS)
    bd_col = find_column(df, POSSIBLE_BD_COLS)

    if stock_col is None:
        raise KeyError(f"Could not find a stock number column in file: {path}")

    # If missing, create empty columns to keep logic simple
    if manager_col is None:
        manager_col = "__manager_missing__"
        df[manager_col] = ""
    if bd_col is None:
        bd_col = "__bd_missing__"
        df[bd_col] = ""

    df[stock_col] = cleaned_series(df[stock_col])
    df[manager_col] = cleaned_series(df[manager_col])
    df[bd_col] = cleaned_series(df[bd_col])

    return df, stock_col, manager_col, bd_col

def get_attribution_set(row: Dict[str, Any], manager_col: str, bd_col: str) -> Set[str]:
    """
    Extracts and cleans manager/BD agent names from a row into a set.
    """
    names = {row[manager_col], row[bd_col]}
    return {name for name in names if name and name not in BAD_SENTINELS}

def main():
    try:
        crm_path = Path(CRM_FILE)
        dms_path = Path(DMS_FILE)

        # Load + prep both files
        crm_df, crm_stock, crm_mgr_col, crm_bd_col = load_and_prepare(crm_path)
        dms_df, dms_stock, dms_mgr_col, dms_bd_col = load_and_prepare(dms_path)

        # Create lookup dictionaries for efficient comparison
        crm_dict = crm_df.drop_duplicates(subset=crm_stock).set_index(crm_stock).T.to_dict('dict')
        dms_dict = dms_df.drop_duplicates(subset=dms_stock).set_index(dms_stock).T.to_dict('dict')

        mismatches = []

        target_manager_normalized = TARGET_MANAGER.strip().lower()

        # Iterate over all unique stock numbers present in either file
        all_stocks = set(crm_dict.keys()).union(set(dms_dict.keys()))

        for stock_num in all_stocks:
            crm_row = crm_dict.get(stock_num)
            dms_row = dms_dict.get(stock_num)

            crm_reps = get_attribution_set(crm_row, crm_mgr_col, crm_bd_col) if crm_row else set()
            dms_reps = get_attribution_set(dms_row, dms_mgr_col, dms_bd_col) if dms_row else set()

            # Check if the target manager is involved in this sale
            if target_manager_normalized in crm_reps or target_manager_normalized in dms_reps:
                # Check for a mismatch in the combined set of attributed reps
                if crm_reps != dms_reps:
                    mismatches.append({
                        "stock_number": stock_num,
                        "crm_attribution": sorted(list(crm_reps)) if crm_reps else "unassigned",
                        "dms_attribution": sorted(list(dms_reps)) if dms_reps else "unassigned",
                    })

        mismatch_df = pd.DataFrame(mismatches)

        print(f"\n=== Mismatches for Manager: {TARGET_MANAGER.title()} by Stock Number ===")
        if not mismatch_df.empty:
            mismatch_df['crm_attribution'] = mismatch_df['crm_attribution'].apply(lambda x: ", ".join(x).title() if isinstance(x, list) else x.title())
            mismatch_df['dms_attribution'] = mismatch_df['dms_attribution'].apply(lambda x: ", ".join(x).title() if isinstance(x, list) else x.title())
            print(mismatch_df.to_string(index=False))
        else:
            print(f"No discrepancies found for {TARGET_MANAGER.title()} between CRM and DMS attribution.")
        print("-----------------------------------------------------")

    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure the input files are uploaded and the names match the configuration.", file=sys.stderr)
    except KeyError as e:
        print(f"Error: A required column was not found in one of the files: {e}", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()



=== Mismatches for Manager: Suna Faitarouny by Stock Number ===
stock_number crm_attribution dms_attribution
     mp10225      Unassigned Suna Faitarouny
     mp10335      Unassigned Suna Faitarouny
-----------------------------------------------------


In [ ]:
#!/usr/bin/env python3
"""
repeatable_missing_rows_finder.py

Task:
- Reads two CSV files specified by the user.
- Compares vehicle stock numbers to find records present in the DMS file
  but missing from the Google Sheet file.
- Outputs the complete missing rows in a format that matches the Google Sheet's
  column order, ready for pasting.

This script is designed for easy use in Google Colab with minimal changes.
"""

import sys
import pandas as pd
from pathlib import Path

# =============================================================================
#
#   USER CONFIGURATION SECTION
#   Change these two variables to point to your input files.
#
# =============================================================================

# Name of the primary sales file (DMS)
DMS_FILE = "/PaytonListtxt.csv"

# Name of the Google Sheet file you want to update
SHEET_FILE = "/August Google sheet  - Main (10).csv"

# =============================================================================
#
#   CORE SCRIPT LOGIC (DO NOT CHANGE BELOW THIS LINE)
#
# =============================================================================

BAD_STOCK_VALUES = {"", "NAN", "NONE", "NULL"}

def normalize_header(col: str) -> str:
    """Normalizes a column header for consistent access."""
    return str(col).strip().lower().replace(" ", "_").replace("#", "number")

def find_stock_col(cols) -> str | None:
    """
    Tries to find the stock number column from a list of headers.
    Returns the normalized column name if found, otherwise None.
    """
    candidates = [
        "vehiclestocknumber", "stock_number", "stocknumber",
        "vehicle_stock_number", "stock", "stock_no", "stocknum"
    ]
    normalized_cols = [normalize_header(c) for c in cols]
    for c in candidates:
        if c in normalized_cols:
            return c
    for c in normalized_cols:
        if "stock" in c:
            return c
    return None

def load_norm(path: Path) -> pd.DataFrame:
    """Loads a CSV and normalizes its headers."""
    df = pd.read_csv(path)
    df.columns = [normalize_header(c) for c in df.columns]
    return df

def build_output(dms_df: pd.DataFrame, sheet_df: pd.DataFrame, sheet_path_for_headers: Path) -> pd.DataFrame:
    """
    Builds the output DataFrame of missing rows,
    formatted to match the Google Sheet's column order.
    """
    dms_stock = find_stock_col(dms_df.columns)
    sheet_stock = find_stock_col(sheet_df.columns)

    if dms_stock is None or sheet_stock is None:
        print("Error: Could not find a stock number column in one or both files. Returning empty DataFrame.", file=sys.stderr)
        sheet_headers_original = list(pd.read_csv(sheet_path_for_headers, nrows=0).columns)
        return pd.DataFrame(columns=sheet_headers_original)

    dms_df["__stock__"] = dms_df[dms_stock].astype(str).str.upper().str.strip()
    sheet_df["__stock__"] = sheet_df[sheet_stock].astype(str).str.upper().str.strip()

    dms_df = dms_df[~dms_df["__stock__"].isin(BAD_STOCK_VALUES)]
    sheet_df = sheet_df[~sheet_df["__stock__"].isin(BAD_STOCK_VALUES)]

    sheet_keys = set(sheet_df["__stock__"])
    missing_keys = [s for s in dms_df["__stock__"].tolist() if s not in sheet_keys]
    if not missing_keys:
        print("No missing stock numbers found. Returning empty DataFrame.", file=sys.stderr)
        return pd.DataFrame(columns=pd.read_csv(sheet_path_for_headers, nrows=0).columns)

    missing_rows = dms_df[dms_df["__stock__"].isin(missing_keys)].copy()

    sheet_headers_original = list(pd.read_csv(sheet_path_for_headers, nrows=0).columns)
    sheet_headers_norm = [normalize_header(c) for c in sheet_headers_original]
    norm_to_orig = {n: o for n, o in zip(sheet_headers_norm, sheet_headers_original)}

    out = pd.DataFrame(columns=sheet_headers_original)
    for norm_col in sheet_headers_norm:
        orig_col = norm_to_orig[norm_col]
        # Check for specific columns and map them from the DMS file
        if norm_col == "sale_date":
            # Explicitly map 'sale_date' from DMS to the output 'Sale Date' column
            if "sale_date" in missing_rows.columns:
                out[orig_col] = missing_rows["sale_date"]
        elif norm_col == "f_&_i":
            if "f_&_i" in missing_rows.columns:
                out[orig_col] = missing_rows["f_&_i"]
            elif "f_&_i_1" in missing_rows.columns:
                out[orig_col] = missing_rows["f_&_i_1"]
        elif norm_col == "f_&_i_2":
            if "f_&_i_2" in missing_rows.columns:
                out[orig_col] = missing_rows["f_&_i_2"]
        elif norm_col in missing_rows.columns:
            out[orig_col] = missing_rows[norm_col]
        elif norm_col == "vehiclestocknumber":
            out[orig_col] = missing_rows[dms_stock].astype(str).str.upper().str.strip()
        else:
            out[orig_col] = ""

    out["__order__"] = missing_rows.index
    out = out.sort_values("__order__", kind="stable").drop(columns="__order__")

    return out

def to_csv_markdown(df: pd.DataFrame) -> str:
    """Converts a DataFrame to a CSV string without index."""
    return df.to_csv(index=False)

if __name__ == "__main__":
    try:
        dms_df = load_norm(Path(DMS_FILE))
        sheet_df = load_norm(Path(SHEET_FILE))

        output_df = build_output(dms_df, sheet_df, Path(SHEET_FILE))

        print("\n" + to_csv_markdown(output_df))

    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure both files are uploaded to your Colab session and the names match exactly.", file=sys.stderr)
    except ValueError as e:
        print(f"Error: {e}", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)


Column 1,salesrepname,splitsalesrep,Manager,BD Agent,F & I,F & I 2,vehiclestocknumber,First Name,Last Name,lead_source,Notes,Column 2
,Tony Durazo,,Suna Faitarouny,,Brent Beck,,MP10319,Michelle,Gill,CARFAX,,
,Youcef Bacha,,,,Brent Beck,,MP9898A,Nicholas,Jordan,CARGURUS,,
,,,,,,,MP9902A,,,,,
,Will Deguire,Robert Emmons,Levi Polege,,Mark Rector,,MP8991,John,Schwieterman,AUTOTRADER,,
,Kevin Claypool,,Suna Faitarouny,,Mark Rector,,MP8793A,Natasha,Juan,CARGURUS,,
,Jordan Lecky,,Suna Faitarouny,,Mark Rector,,MP9794,Hugh,Thompson,AUTOTRADER,,
,Tony Durazo,,Suna Faitarouny,,Yesenia Felix,,MP10063,Dede,Moore,DEALERS WEBSITE,,
,Robert Emmons,,Suna Faitarouny,,Mark Rector,,MP10065,Braden,Peters,,,
,Jordan Lecky,,Suna Faitarouny,,Yesenia Felix,,MP10074,O'Neil,Hersman,CARGURUS,,
,Youcef Bacha,Tony Durazo,Suna Faitarouny,,Yesenia Felix,,MP10121,Mark,Shilton,,,
",,
,,,,,,,MP9826A,Brandon,Dunlap,,,



In [ ]:
#!/usr/bin/env python3
"""
repeatable_missing_rows_finder.py

Output schema (exact order):
sale_date, salesrepname, splitsalesrep, manager, bd_agent,
f_&_i, f_&_i_2, vehiclestocknumber, first_name, last_name, lead_source
"""

import sys, csv
import pandas as pd
from pathlib import Path
from typing import Iterable, Optional

# ===================== USER CONFIG =====================
DMS_FILE = "/PaytonListtxt.csv"                  # e.g., DeskManager / DMS export
SHEET_FILE = "/content/August Google sheet  - Main.csv"  # your Google Sheet export
# ======================================================

BAD_STOCK_VALUES = {"", "NAN", "NONE", "NULL"}

OUTPUT_COLUMNS = [
    "sale_date", "salesrepname", "splitsalesrep", "manager", "bd_agent",
    "f_&_i", "f_&_i_2", "vehiclestocknumber", "first_name", "last_name", "lead_source"
]

COLUMN_VARIANTS = {
    "sale_date":        ["sale_date", "sale date", "date", "date_of_sale", "sold_date"],
    "salesrepname":     ["salesrepname", "sales_rep_name", "sales_rep", "rep", "salesperson"],
    "splitsalesrep":    ["splitsalesrep", "split_sales_rep", "splitrep", "split_salesperson"],
    "manager":          ["manager", "sales_manager", "desk_manager"],
    "bd_agent":         ["bd_agent", "bdagent", "bd", "bd_agent_name", "bd rep"],
    "f_&_i":            ["f_&_i", "f & i", "f_i", "finance_manager", "fni", "f and i"],
    "f_&_i_2":          ["f_&_i_2", "f & i 2", "f_i_2", "finance_manager_2", "fni_2", "f and i 2"],
    "vehiclestocknumber": ["vehiclestocknumber", "vehicle_stock_number", "stock_number", "stocknumber", "stock", "stock_no", "stocknum"],
    "first_name":       ["first_name", "first name", "fname", "customer_first_name"],
    "last_name":        ["last_name", "last name", "lname", "customer_last_name", "surname"],
    "lead_source":      ["lead_source", "lead source", "source", "leadsource"]
}

def normalize_header(col: str) -> str:
    return str(col).strip().lower().replace(" ", "_").replace("#", "number")

def load_norm(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = [normalize_header(c) for c in df.columns]
    return df

def find_first(df: pd.DataFrame, candidates: Iterable[str]) -> Optional[str]:
    normalized = [normalize_header(c) for c in candidates]
    df_cols = set(df.columns)
    for c in normalized:
        if c in df_cols:
            return c
    return None

def find_stock_col(cols) -> Optional[str]:
    candidates = [
        "vehiclestocknumber", "stock_number", "stocknumber",
        "vehicle_stock_number", "stock", "stock_no", "stocknum"
    ]
    normalized_cols = [normalize_header(c) for c in cols]
    for c in candidates:
        if c in normalized_cols:
            return c
    for c in normalized_cols:
        if "stock" in c:
            return c
    return None

def clean_strings(df: pd.DataFrame) -> pd.DataFrame:
    """Blank out NaNs/'nan', trim whitespace, normalize newlines, drop quote-only rows."""
    df = df.copy().where(df.notna(), "")
    for c in df.columns:
        df[c] = (
            df[c].astype(str)
                 .str.replace(r"^\s*nan\s*$", "", regex=True)
                 .str.replace("\r\n", "\n")
                 .str.replace("\r", "\n")
                 .str.strip()
        )
    # drop rows that collapse to just a single double-quote
    mask_bad = df.apply(lambda r: "".join(r.values.astype(str)).strip() == '"', axis=1)
    return df[~mask_bad]

def build_output(dms_df: pd.DataFrame, sheet_df: pd.DataFrame) -> pd.DataFrame:
    dms_stock = find_stock_col(dms_df.columns)
    sheet_stock = find_stock_col(sheet_df.columns)
    if dms_stock is None or sheet_stock is None:
        raise ValueError("Could not find a stock number column in one or both files.")

    dms_df["__stock__"] = dms_df[dms_stock].astype(str).str.upper().str.strip()
    sheet_df["__stock__"] = sheet_df[sheet_stock].astype(str).str.upper().str.strip()

    dms_df = dms_df[~dms_df["__stock__"].isin(BAD_STOCK_VALUES)]
    sheet_df = sheet_df[~sheet_df["__stock__"].isin(BAD_STOCK_VALUES)]

    sheet_keys = set(sheet_df["__stock__"])
    missing_keys = [s for s in dms_df["__stock__"].tolist() if s not in sheet_keys]
    if not missing_keys:
        return pd.DataFrame(columns=OUTPUT_COLUMNS)

    missing_rows = dms_df[dms_df["__stock__"].isin(missing_keys)].copy()
    missing_rows["__order__"] = range(len(missing_rows))

    out = pd.DataFrame(index=missing_rows.index, columns=OUTPUT_COLUMNS, dtype=object)
    for out_col in OUTPUT_COLUMNS:
        if out_col == "vehiclestocknumber":
            out[out_col] = missing_rows["__stock__"]
            continue
        src_col = find_first(missing_rows, COLUMN_VARIANTS[out_col])
        out[out_col] = missing_rows[src_col] if src_col else ""

    out["__order__"] = missing_rows["__order__"]
    out = out.sort_values("__order__", kind="stable").drop(columns="__order__")

    # force strings
    for c in OUTPUT_COLUMNS:
        out[c] = out[c].astype(str)

    return clean_strings(out)

def main():
    try:
        dms_df = load_norm(Path(DMS_FILE))
        sheet_df = load_norm(Path(SHEET_FILE))
        output_df = build_output(dms_df, sheet_df)

        # Paste-friendly TSV (optional file)
        output_df.to_csv("missing_rows.tsv", sep="\t", index=False, lineterminator="\n")

        # Clean CSV (fully quoted for Import)
        output_df.to_csv(
            "missing_rows.csv",
            index=False,
            lineterminator="\n",
            quoting=csv.QUOTE_ALL,
            escapechar="\\",
            na_rep=""
        )

        # Console preview (CSV, normal-looking)
        print(output_df.to_csv(index=False, lineterminator="\n"))
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure both files are uploaded and paths are correct.", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()



sale_date,salesrepname,splitsalesrep,manager,bd_agent,f_&_i,f_&_i_2,vehiclestocknumber,first_name,last_name,lead_source
8/11/25,,,,,,,MP9880A,WILLIAM,DEGUIRE,
8/11/25,,,,,,,MP10059A,KURT,JENSEN,
8/11/25,,,,,,,MP9181A,KURT,JENSEN,
8/13/25,Brandon Beltran,Tony Durazo,Jamie Sheridan,,Mark Rector,,MP10217,Teresa,Blackhair,CARS.COM

